<a href="https://colab.research.google.com/github/MamedenQ/OpenPoseExample/blob/main/OpenPose%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%9F%E5%8B%95%E4%BD%9C%E8%A7%A3%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境確認

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

# マウント

In [ ]:
# Google Drive のマウント（Google Drive のファイルを読み書きできるようにする）
from google.colab import drive
drive.mount('/content/drive')

# OpenPoseのインストール

In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

# 動画から動作解析

## 1.事前設定

解析元動画情報入力

In [ ]:
# trueの場合はmovからmp4へ形式変換が行われる
IS_MOV_FORMAT = True

# 動画のファイル名を設定すること
MOVIE_NAME = 'm3.mov'
MOVIES_DIR = '/content/drive/MyDrive/Colaboratory/openpose/movies'

## 2.解析

### 2.1.動画準備

MOV>MP4変換、または、動画コピー

In [ ]:
!cd /content

!rm analyze_movie.mp4

if IS_MOV_FORMAT:
    !ffmpeg -i {MOVIES_DIR}/{MOVIE_NAME} analyze_movie.mp4
else:
    !cp {MOVIES_DIR}/{MOVIE_NAME} .
    !mv {MOVIE_NAME} analyze_movie.mp4

### 2.2.解析

In [ ]:
!cd /content

!rm -rf json
!mkdir json

!rm openpose.avi

!cd openpose && ./build/examples/openpose/openpose.bin --video ../analyze_movie.mp4 --write_json ../json --display 0 --write_video ../openpose.avi

AVI>MP4変換

In [ ]:
!cd /content
!rm analyzed.mp4

!ffmpeg -y -loglevel info -i openpose.avi analyzed.mp4

解析動画表示

In [ ]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('analyzed.mp4', width=960, height=720)

### 2.3.解析済み動画コピー

In [ ]:
!cp /content/analyzed.mp4 {MOVIES_DIR}

# 画像から動作解析

## 解析

事前設定

In [ ]:
# 画像格納ディレクトリ指定
IMAGES_DIR = '/content/drive/MyDrive/Colaboratory/openpose/images'

解析

In [ ]:
!cd /content

JSON_DIR = IMAGES_DIR + '/json'
OUTPUT_DIR = IMAGES_DIR + '/output'

!rm -rf {JSON_DIR}
!mkdir {JSON_DIR}

!rm -rf {OUTPUT_DIR}
!mkdir {OUTPUT_DIR}

!cd openpose && ./build/examples/openpose/openpose.bin --image_dir {IMAGES_DIR} --write_json {JSON_DIR} --display 0 --write_images {OUTPUT_DIR}